In [2]:
import heapq
import math

class RoadNetwork:
    def __init__(self, constant_multiplier=1.5):
        self.graph = {}
        self.constant_multiplier = constant_multiplier

    def add_segment(self, start, end, max_speed, gradient):
        """ Adds a directed road segment with expected travel duration """
        distance = math.dist(start, end)  # Real-world distance calculation
        duration = gradient * self.constant_multiplier * (distance / max_speed)

        if start not in self.graph:
            self.graph[start] = []
        self.graph[start].append((end, duration))

    def dijkstra(self, start, target):
        """ Finds shortest path using Dijkstra's algorithm with expected travel duration """
        pq = [(0, start)]  # Priority queue with (cost, node)
        durations = {node: float('inf') for node in self.graph}
        durations[start] = 0
        previous_nodes = {}

        while pq:
            current_duration, current_node = heapq.heappop(pq)

            if current_node == target:
                break

            for neighbor, duration in self.graph.get(current_node, []):
                total_duration = current_duration + duration
                if total_duration < durations[neighbor]:
                    durations[neighbor] = total_duration
                    heapq.heappush(pq, (total_duration, neighbor))
                    previous_nodes[neighbor] = current_node

        path = []
        node = target
        while node in previous_nodes:
            path.insert(0, node)
            node = previous_nodes[node]
        if path:
            path.insert(0, start)

        return path if path else None

# Example: Setting up a real-world mine road network
road_network = RoadNetwork()

# Adding road segments with real-world distances (kilometers), max speeds (km/h), and gradients
road_network.add_segment((0,0), (3,0), max_speed=80, gradient=1.05)  # Straight flat road
road_network.add_segment((3,0), (6,1), max_speed=60, gradient=1.10)  # Slight incline
road_network.add_segment((6,1), (9,3), max_speed=50, gradient=1.20)  # Steeper section

# Reverse direction carriageway
road_network.add_segment((9,3), (6,1), max_speed=50, gradient=1.15)
road_network.add_segment((6,1), (3,0), max_speed=60, gradient=1.05)
road_network.add_segment((3,0), (0,0), max_speed=80, gradient=1.00)

# Intersection connections considering left-hand drive movement
road_network.add_segment((3,0), (3,-1), max_speed=30, gradient=1.02)  # Left turn
road_network.add_segment((3,-1), (6,-1), max_speed=50, gradient=1.07)
road_network.add_segment((6,-1), (6,1), max_speed=40, gradient=1.05)  # Rejoining

# Finding the shortest travel duration path
start_point = (0,0)
end_point = (9,3)
shortest_path = road_network.dijkstra(start_point, end_point)
print("Shortest Path:", shortest_path)


Shortest Path: [(0, 0), (3, 0), (6, 1), (9, 3)]


In [2]:
# Using Labels for nodes
import heapq
import math

class RoadNetwork:
    def __init__(self, constant_multiplier=1.5):
        self.graph = {}
        self.constant_multiplier = constant_multiplier

    def add_segment(self, start_label, end_label, max_speed, gradient, start_coords, end_coords):
        """ Adds a directed road segment with expected travel duration based on real-world distances """
        distance = math.dist(start_coords, end_coords)  # Compute distance in kilometers
        duration = gradient * self.constant_multiplier * (distance / max_speed)  # Expected travel duration

        if start_label not in self.graph:
            self.graph[start_label] = []
        self.graph[start_label].append((end_label, duration))

    def dijkstra(self, start_label, target_label):
        """ Finds shortest path using Dijkstra's algorithm with expected travel duration """
        pq = [(0, start_label)]  # Priority queue with (cost, node)
        durations = {node: float('inf') for node in self.graph}
        durations[start_label] = 0
        previous_nodes = {}

        while pq:
            current_duration, current_node = heapq.heappop(pq)

            if current_node == target_label:
                break

            for neighbor, duration in self.graph.get(current_node, []):
                total_duration = current_duration + duration
                if total_duration < durations[neighbor]:
                    durations[neighbor] = total_duration
                    heapq.heappush(pq, (total_duration, neighbor))
                    previous_nodes[neighbor] = current_node

        path = []
        node = target_label
        while node in previous_nodes:
            path.insert(0, node)
            node = previous_nodes[node]
        if path:
            path.insert(0, start_label)

        return path if path else None

# Example: Defining a mine road network with labeled points
road_network = RoadNetwork()

# Adding road segments with labeled start/end points, real-world distances (km), max speeds (km/h), and gradients
road_network.add_segment("Entrance", "Main Road", max_speed=80, gradient=1.05, start_coords=(0, 0), end_coords=(3, 0))
road_network.add_segment("Main Road", "Hill Base", max_speed=60, gradient=1.10, start_coords=(3, 0), end_coords=(6, 1))
road_network.add_segment("Hill Base", "Mine Site", max_speed=50, gradient=1.20, start_coords=(6, 1), end_coords=(9, 3))

# Reverse direction carriageway
road_network.add_segment("Mine Site", "Hill Base", max_speed=50, gradient=1.15, start_coords=(9, 3), end_coords=(6, 1))
road_network.add_segment("Hill Base", "Main Road", max_speed=60, gradient=1.05, start_coords=(6, 1), end_coords=(3, 0))
road_network.add_segment("Main Road", "Entrance", max_speed=80, gradient=1.00, start_coords=(3, 0), end_coords=(0, 0))

# Intersection handling (left-hand drive movement)
road_network.add_segment("Main Road", "Side Road", max_speed=30, gradient=1.02, start_coords=(3, 0), end_coords=(3, -1))
road_network.add_segment("Side Road", "Hill Base Side", max_speed=50, gradient=1.07, start_coords=(3, -1), end_coords=(6, -1))
road_network.add_segment("Hill Base Side", "Hill Base", max_speed=40, gradient=1.05, start_coords=(6, -1), end_coords=(6, 1))

# Finding shortest path based on expected travel duration
start_label = "Entrance"
end_label = "Mine Site"
shortest_path = road_network.dijkstra(start_label, end_label)
print("Shortest Path:", shortest_path)


Shortest Path: ['Entrance', 'Main Road', 'Hill Base', 'Mine Site']


In [4]:
# use visited nodes tracking to prevent unecessary searches in loops such as ring road.
import heapq
import math

class RoadNetwork:
    def __init__(self, constant_multiplier=1.5):
        self.graph = {}
        self.constant_multiplier = constant_multiplier

    def add_segment(self, start_label, end_label, max_speed, gradient, start_coords, end_coords):
        """ Adds a directed road segment with expected travel duration """
        distance = math.dist(start_coords, end_coords)  # Compute real-world distance
        duration = gradient * self.constant_multiplier * (distance / max_speed)  # Compute travel duration

        if start_label not in self.graph:
            self.graph[start_label] = []
        self.graph[start_label].append((end_label, duration))

    def dijkstra(self, start_label, target_label):
        """ Finds shortest path using Dijkstra's algorithm with cycle prevention """
        pq = [(0, start_label)]  # Priority queue with (cost, node)
        durations = {node: float('inf') for node in self.graph}
        durations[start_label] = 0
        previous_nodes = {}
        visited = set()  # Track processed nodes

        while pq:
            current_duration, current_node = heapq.heappop(pq)

            if current_node in visited:
                continue  # Skip re-processing completed nodes
            
            visited.add(current_node)

            if current_node == target_label:
                break

            for neighbor, duration in self.graph.get(current_node, []):
                total_duration = current_duration + duration
                if total_duration < durations[neighbor]:
                    durations[neighbor] = total_duration
                    heapq.heappush(pq, (total_duration, neighbor))
                    previous_nodes[neighbor] = current_node

        path = []
        node = target_label
        while node in previous_nodes:
            path.insert(0, node)
            node = previous_nodes[node]
        if path:
            path.insert(0, start_label)

        return path if path else None

# Example: Setting up a ring road in a mine network
road_network = RoadNetwork()

road_network.add_segment("A", "B", max_speed=80, gradient=1.05, start_coords=(0, 0), end_coords=(3, 0))
road_network.add_segment("B", "C", max_speed=60, gradient=1.10, start_coords=(3, 0), end_coords=(6, 1))
road_network.add_segment("C", "D", max_speed=50, gradient=1.20, start_coords=(6, 1), end_coords=(9, 3))
road_network.add_segment("D", "A", max_speed=80, gradient=1.00, start_coords=(9, 3), end_coords=(0, 0))  # Closing the ring

# Finding shortest path between two points on the ring road
start_label = "A"
end_label = "C"
shortest_path = road_network.dijkstra(start_label, end_label)
print("Shortest Path:", shortest_path)


Shortest Path: ['A', 'B', 'C']


Sure! Here’s a **step-by-step explanation** of how **Dijkstra’s Algorithm** works:

### **1. Initialization**
- Choose a **starting node** (where you begin your journey).
- Set the **cost to reach the start node to `0`**.
- Set the cost to reach all other nodes to **infinity (`∞`)** (since we haven’t calculated their shortest paths yet).
- Create a **priority queue** (or a min heap) to store nodes **sorted by their current shortest known distance**.

### **2. Process the Start Node**
- Start with the **node having the smallest known distance** (initially the start node).
- Mark it as **visited**—it won’t be revisited because we have finalized its shortest path.
- Look at all **neighboring nodes** connected to this node and calculate the cost to reach them.

### **3. Update Neighboring Nodes**
- For each **neighbor** of the current node:
  - **Compute the new potential shortest distance**:  
    `distance to current node + edge cost to neighbor`
  - If the new calculated distance is **shorter than the previously known distance**, update it.
  - Store which node led to this shortest distance (for reconstructing the path later).
  - Add this neighbor to the **priority queue** for future processing.

### **4. Select the Next Node**
- Remove the **node with the smallest known distance** from the priority queue.
- Repeat **steps 2 & 3** for this node.

### **5. Stop When Target Node is Processed**
- If we reach the **target destination**, stop processing further nodes.
- Otherwise, continue selecting the next smallest distance node and repeating the process.

### **6. Reconstruct the Shortest Path**
- **Trace back** from the target node using stored previous nodes to reconstruct the optimal path.
- The result is the **shortest route** from the start node to the destination.

### **Key Properties of Dijkstra’s Algorithm**
✔ **Greedy Choice** – Always picks the node with the smallest known distance first.  
✔ **No Cycles / Infinite Loops** – Ensures that nodes are **not revisited** once their shortest path is finalized.  
✔ **Guarantees Optimality** – Works best with **non-negative** edge weights, ensuring **always the shortest path**.  

Would you like a **visual example** or some **interactive code** to help illustrate it further? 🚦😃


In [4]:
import math
import numpy as np
math.dist(np.array([445.6, 689.1]), np.array([453.6, 687.5]))

8.15843122174846